## Data Gathering

In [1]:
#Load library
import pandas as pd 
import numpy as np
import requests
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
%matplotlib inline

### Load archived twitter data from csv file

In [2]:
#Read @WeRateDogs twitter acrhive 
df_archive = pd.read_csv("twitter-archive-enhanced.csv")

In [3]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
df_archive.tweet_id[1]

892177421306343426

### Load more information using twitter apis

In [5]:
#Connect Twitter APIs to query more data related to archived tweets
#KEYS HIDDEN

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [32]:
df_archive.shape[0]

2356

In [8]:
fails_dict = {}
tweet_info_bulk = []
for n in range(0,df_archive.shape[0]+100,100):
    if df_archive.shape[0] - n < 100:
        tweet_id = df_archive.tweet_id[n:df_archive.shape[0]].tolist()
    else:
        tweet_id = df_archive.tweet_id[n:n+100].tolist()
    try:
        lookups = api.statuses_lookup(id_=tweet_id)
        for info in lookups:
            tweet_info_bulk.append({"tweet_id": info.id,
                            "favorite_count": info.favorite_count,
                            "content": info.text})
    except tweepy.TweepError as e:
        fails_dict[e] = tweet_id
#     except tweepy.TweepError as e:
#         print("fail")
#         fails_dict[tweet.id] = e

In [9]:
with open("./tweet_info.json", "w+") as file:
    json.dump(tweet_info_bulk, file)

In [10]:
df_tweet_info = pd.read_json("./tweet_info.json")

In [11]:
df_tweet_info.shape

(2332, 3)

In [12]:
df_tweet_info.head()

,content,favorite_count,tweet_id
0,This is Benedict. He wants to thank you for th...,18025,876484053909872640
1,This is Roscoe. Another pupper fallen victim t...,20274,886366144734445568
2,RT @rachel2195: @dog_rates the boyfriend and h...,0,877611172832227328
3,This is Gerald. He was just told he didn't get...,20852,888078434458587136
4,This is Darla. She commenced a snooze mid meal...,40373,891689557279858688


### Load image prediction data from tsv file 

In [13]:
df_img_predict = pd.read_csv("image-predictions.tsv", delimiter='\t')

## Data Assessing 

In [14]:
df_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1406,699088579889332224,NaN,NaN,2016-02-15 04:31:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charl. He's a bully. Chucks that dumbb...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/699088579...,3,10,Charl,None,None,None,None
1804,676946864479084545,NaN,NaN,2015-12-16 02:08:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pups goal was to get all four feet as clo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676946864...,12,10,None,None,None,None,None
2240,667924896115245057,NaN,NaN,2015-11-21 04:37:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jiminy. He has always wanted to be a c...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667924896...,9,10,Jiminy,None,None,None,None
2237,668142349051129856,NaN,NaN,2015-11-21 19:02:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This lil pup is Oliver. Hops around. Has wings...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668142349...,2,10,None,None,None,None,None
162,860524505164394496,NaN,NaN,2017-05-05 16:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Carl. He likes to dance. Doesn't care ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/860524505...,13,10,Carl,None,None,None,None
934,753420520834629632,NaN,NaN,2016-07-14 02:47:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we are witnessing an isolated squad of bo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/753420520...,11,10,None,None,None,None,None
1674,682259524040966145,NaN,NaN,2015-12-30 17:58:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He's an Iglesias Hufflepoof. Quite t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682259524...,9,10,Jax,None,None,None,None
1037,744709971296780288,NaN,NaN,2016-06-20 01:54:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Harvey. He's stealthy af. 10/10 would ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/744709971...,10,10,Harvey,None,None,None,None
2174,669216679721873412,NaN,NaN,2015-11-24 18:11:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Clarq. He's a golden Quetzalcoatl. Cla...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669216679...,8,10,Clarq,None,None,None,None
1212,715342466308784130,NaN,NaN,2016-03-31 00:58:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oscar. He's a world renowned snowball ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715342466...,10,10,Oscar,None,None,None,None


In [15]:
df_img_predict.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [16]:
df_tweet_info.head()

,content,favorite_count,tweet_id
0,This is Benedict. He wants to thank you for th...,18025,876484053909872640
1,This is Roscoe. Another pupper fallen victim t...,20274,886366144734445568
2,RT @rachel2195: @dog_rates the boyfriend and h...,0,877611172832227328
3,This is Gerald. He was just told he didn't get...,20852,888078434458587136
4,This is Darla. She commenced a snooze mid meal...,40373,891689557279858688


### Data Quality Issues

#### df_archive 
- There are some retweet data without dog ratings
- Lots of unnecessary columns from "archive data' such as ["in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","source","expanded_urls"]
- Numerator 13.5 is extracted as 5 and data type is integer
- Extract the number of legs as rating e.g. 3 1/2 legged 
- Dog names are not properly extracted e.g. a, the, an
- "Twitter_id" datatype is Integer
- Timestamp colum is not "datetime" type

#### df_img_predict

- Lettercase is not consistent in p1, p2, p3 columns of image prediction table

### Data Tideness Issues 
- All 3 dataframes are seperate
- Dog stage values are scattered over 4 columns ['doggo','floofer','pupper','puppo']


## Data Cleaning 

In [154]:
#create a copy for each dataframe
archive_clean = df_archive.copy()
tweet_info_clean = df_tweet_info.copy()
img_predict_clean = df_img_predict.copy()

### Define
Only include the rows without any values in in_reply_to_status_id or retweeted_status_id


### Code

In [155]:
archive_clean = archive_clean[archive_clean['in_reply_to_status_id'].isnull()] 
archive_clean = archive_clean[archive_clean['retweeted_status_id'].isnull()]

### Test

In [19]:
sum(archive_clean['in_reply_to_status_id'].isnull()) == archive_clean.shape[0] and sum(archive_clean['retweeted_status_id'].isnull()) == archive_clean.shape[0]

True

### Define
Drop unnessesary columns ("in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","source","expanded_urls") from archive_clean


In [156]:
archive_clean = archive_clean.drop(["in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","source","expanded_urls"],axis=1)

### Test


In [21]:
columns = ["in_reply_to_status_id","in_reply_to_user_id","retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp","source","expanded_urls"]
pd.Series(columns).isin(archive_clean.columns).all()

False

### Define
Change numerator and denominator datatypes to float and extract the data from the text column using regex. 


In [22]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(7)
memory usage: 180.2+ KB


In [23]:
#find rows with decimal dog rating
archive_clean[archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

/Users/uyujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
45,883482846933004288,2017-07-08 00:28:19 +0000,This is Bella. She hopes her smile made you sm...,5,10,Bella,None,None,None,None
695,786709082849828864,2016-10-13 23:23:56 +0000,"This is Logan, the Chow who lived. He solemnly...",75,10,Logan,None,None,None,None
763,778027034220126208,2016-09-20 00:24:34 +0000,This is Sophie. She's a Jubilant Bush Pupper. ...,27,10,Sophie,None,None,pupper,None
1712,680494726643068929,2015-12-25 21:06:00 +0000,Here we have uncovered an entire battalion of ...,26,10,None,None,None,None,None


In [157]:
#replace the integar rating_numerator with decimal rating_numerator extracted from the text again 
archive_clean['rating_numerator'].loc[archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")] = archive_clean[archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")].text.str.extract(r"(\d+\.\d*)").astype('float')[0]


/Users/uyujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/uyujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


### Test


In [59]:
archive_clean[archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]['rating_numerator']

/Users/uyujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


45      13.50
695      9.75
763     11.27
1712    11.26
Name: rating_numerator, dtype: float64

In [163]:
#change the data type of denomiator to float as well
archive_clean.rating_denominator = archive_clean.rating_denominator.astype('float')

### Define
Find the row with 1/2 rating and replace with the actual rating manually

In [164]:
archive_clean[(archive_clean.rating_numerator == 1) & (archive_clean.rating_denominator == 2)]

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [159]:
pd.set_option('display.max_colwidth', -1)
archive_clean[(archive_clean.rating_numerator == 1) & (archive_clean.rating_denominator == 2)].text

2335    This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv
Name: text, dtype: object

In [165]:
archive_clean.rating_numerator.loc[archive_clean.text.str.contains("3 1/2")] = 9

In [166]:
archive_clean.rating_denominator.loc[archive_clean.text.str.contains("3 1/2")] = 10

### Test


In [167]:
archive_clean[archive_clean.text.str.contains("3 1/2")]

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2335,666287406224695296,2015-11-16 16:11:11 +0000,This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,9.0,10.0,an,None,None,None,None


### Define
Replace dog names that's not extracted properly with None

In [112]:
archive_clean[['name', 'text']]

,name,text
0,Phineas,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU
1,Tilly,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV"
2,Archie,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB
3,Darla,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ
4,Franklin,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f"
5,None,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh
6,Jax,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl
7,None,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq
8,Zoey,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b
9,Cassie,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A


In [168]:
archive_clean.name.loc[archive_clean.name.str.islower()] = "None"

### Test


In [131]:
assert all(archive_clean.loc[archive_clean.name.str.islower()] == 0) == True

### Define
- change the datatype of tweet_id to string
- change the datatype of datetime to datetime format

In [233]:
archive_clean.tweet_id = archive_clean.tweet_id.astype("str")
img_predict_clean.tweet_id = archive_clean.tweet_id.astype("str")
tweet_info_clean.tweet_id = archive_clean.tweet_id.astype("str")

In [170]:
archive_clean.timestamp = archive_clean.timestamp.astype("datetime64[ns]")

### Test


In [172]:
archive_clean.tweet_id.dtype, archive_clean.timestamp.dtype

(dtype('O'), dtype('<M8[ns]'))

### Define
Capitalize the string in p1, p2, p3 columns

In [229]:
img_predict_clean.p1 = img_predict_clean.p1.str.capitalize()
img_predict_clean.p2 = img_predict_clean.p2.str.capitalize()
img_predict_clean.p3 = img_predict_clean.p3.str.capitalize()

### Test


In [230]:
img_predict_clean.p1[0][0].isupper()

True

### Define
Replace None with empty string in [doggo, floofer, pupper, puppo] columns and merge the columns together

In [203]:
stages = ['doggo', 'floofer', 'pupper', 'puppo']
for stage in stages:
    archive_clean[stage].replace("None", "", inplace=True)
    archive_clean['stage'] =+ archive_clean[stage]

In [226]:
archive_clean.drop(stages, axis=1, inplace=True)

### Test


In [227]:
set(stages).issubset(archive_clean.columns)

False

### Define
Merge archive_clean, img_predict_clean, and tweet_info_clean

In [235]:
merged_2 = pd.merge(archive_clean, img_predict_clean, on="tweet_id", how="inner")

In [236]:
wrd_master = pd.merge(merged_2, tweet_info_clean, on="tweet_id", how="inner")

### Test


In [239]:
sum(wrd_master['tweet_id'].duplicated())

0

### Define
Add rating column to wrd_master

In [241]:
wrd_master['rating'] = wrd_master['rating_numerator'] / wrd_master['rating_denominator']

### Test


In [242]:
wrd_master['rating']

0       1.3 
1       1.3 
2       1.2 
3       1.3 
4       1.2 
5       1.3 
6       1.3 
7       1.3 
8       1.3 
9       1.4 
10      1.3 
11      1.3 
12      1.3 
13      1.2 
14      1.3 
15      1.3 
16      1.2 
17      1.3 
18      1.3 
19      1.2 
20      1.3 
21      1.4 
22      1.3 
23      1.3 
24      1.2 
25      1.3 
26      1.3 
27      1.3 
28      1.2 
29      1.3 
       ...  
1792    1.0 
1793    1.0 
1794    0.9 
1795    0.8 
1796    1.1 
1797    1.1 
1798    0.8 
1799    1.0 
1800    1.1 
1801    0.7 
1802    1.1 
1803    0.9 
1804    1.0 
1805    0.9 
1806    1.0 
1807    1.2 
1808    1.0 
1809    0.8 
1810    0.5 
1811    1.1 
1812    1.2 
1813    0.9 
1814    0.9 
1815    1.0 
1816    0.8 
1817    0.4 
1818    0.6 
1819    0.8 
1820    1.2 
1821    42.0
Name: rating, Length: 1822, dtype: float64